<a href="https://colab.research.google.com/github/KevinLolochum/BERT-MODELS/blob/main/BART_Fine_tuned_for_Sentiment_Analysis_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INTRODUCTION**
- **Sentiment Analysis**  is the task of classifying text into a certain category.
- The most common categories are happy or sad, spam or not, offensive or not. -
- But sentiment analysis can also have multiple class like happy, neutral, and neutral.
- As discussed in the explanation **BART** is one of the transformer models that is very good at sentiment analysis as I hope to demonstrate in this example.

In [35]:
# Code for checking the amount of RAM that you have.

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


1. **Importing transformer models and libraries**

In [ ]:
!pip install transformers

In [37]:
# Libraries and transformers models
import torch
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm.notebook import tqdm
from transformers import BartTokenizer, BartForSequenceClassification


3. **Loading the data and preprocessing**

- The data I am using in this example is the psy data which requires virtual no preprocessing as I had processed it in an ealier task.
- The psy dataset is a youtube comments dataset that classifies comments as either spam or not.
- Feel free to use a dataset of your choice for this task.


In [ ]:
# Loading the dataset

from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
 df = pd.read_csv('/content/gdrive/My Drive/psy.csv')
 df.head(2)

In [40]:
# Split using sklearn, only selecting a small portion of the data
from sklearn.model_selection import train_test_split

train_df, valid_df = train_test_split(df, random_state = 42, train_size = 0.7, stratify = df.CLASS.values)
train_df.shape

(244, 5)

In [41]:
possible_labels = train_df.CLASS.unique()
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

4. **Tokenizing the data**

- Bart tokenizer takes sentences and returns input_ids, attention_masks. Does not require token_type_type_ids therefore no point returning them.
- Decoder_input_ids which the model requires will be created by the model if not provided.

In [42]:
# Creating model and tokenizer
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

In [43]:
# Tokenizing the questions in the dataset

encoded_data_train = tokenizer.batch_encode_plus( 
    train_df.CONTENT.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding = 'max_length',
    truncation = True,
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    valid_df.CONTENT.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding = 'max_length',
    truncation = True,
    max_length=256, 
    return_tensors='pt'
)

# Model inputs

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_df.CLASS.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(valid_df.CLASS.values)

In [44]:
from torch.utils.data import TensorDataset

# Parsing inputs through the TensorDaset generator.

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [45]:
# Creating a data looader and specifying sampler.

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 20

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

5. **Model initialization**

In [ ]:
model = BartForSequenceClassification.from_pretrained('facebook/bart-base',
                                                      num_labels=len(train_df.CLASS.unique()),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Data scheduler and optimizer

In [47]:
# Optimizer and data scheduler
from transformers import AdamW, get_linear_schedule_with_warmup

epochs = 10

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


Defining performance metrics

In [48]:
from sklearn.metrics import f1_score

# defining the accuracy score
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

# Accuracy per class
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

6. **Training**

In [49]:
# Seed for generating random data

import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [52]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [53]:
# Model training and evaluation

torch.cuda.empty_cache

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BART_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {round(loss_train_avg, 2)}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {round(val_loss,2)}')
    tqdm.write(f'F1 Score (Weighted): {round(val_f1, 2)}')


Epoch 1
Training loss: 0.7
Validation loss: 0.64
F1 Score (Weighted): 0.63



Epoch 2
Training loss: 0.59
Validation loss: 0.49
F1 Score (Weighted): 0.9



Epoch 3
Training loss: 0.46
Validation loss: 0.29
F1 Score (Weighted): 0.95



Epoch 4
Training loss: 0.27
Validation loss: 0.16
F1 Score (Weighted): 0.98



Epoch 5
Training loss: 0.17
Validation loss: 0.09
F1 Score (Weighted): 0.97



Epoch 6
Training loss: 0.11
Validation loss: 0.07
F1 Score (Weighted): 0.98



Epoch 7
Training loss: 0.05
Validation loss: 0.05
F1 Score (Weighted): 0.99



Epoch 8
Training loss: 0.04
Validation loss: 0.04
F1 Score (Weighted): 0.99



Epoch 9
Training loss: 0.04
Validation loss: 0.05
F1 Score (Weighted): 0.98



Epoch 10
Training loss: 0.04
Validation loss: 0.05
F1 Score (Weighted): 0.98



In [24]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [25]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 52/53

Class: 1
Accuracy: 53/53



BART Model does very well as can be shown by number of correct predictions. Achieves almost 100% in determining whether a comment is spam(1) or not (0) 